<a href="https://colab.research.google.com/github/aegoff/Coursera_Capstone/blob/main/IBM3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Research Question: Where in Columbus, OH is the best place to set up a high-end British restaurant?

Audience: Entrepreneurs in the restaurant industry who want to know the best place to establish a restaurant

Methodology:


1.   Webscrape info about mediam household income, population, and population-density from "UnitedStatesZipCode.org"
2.   Webscrape longitudes and latitudes corresponding to zipcodes from Eric Hurst's Github repository
3.   Collect venue data from Foursquare via API
4.   Use the one-hot encoding technique which converts the categorical values into dummies so they can be used for machine learning.
5.   Begin the clustering process (K-means)
6.   Visualize with Folium
7.   Answer Research Quetions


First, we need to prepare the dataframe

In [1]:
!pip install html-table-parser-python3
!pip install gazpacho
!pip install bs4
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
from pprint import pprint
from html_table_parser.parser import HTMLTableParser
import pandas as pd
from gazpacho import Soup

zipcodes= [43002, 43004, 43016, 43017, 43026, 43035, 43054, 43065, 43081, 43082, 43085, 43119, 43123, 43137, 43147, 43201, 43202, 43203, 43204, 43205, 43206, 43207, 43210, 43211, 43212, 43213, 43214, 43215, 43217, 43219, 43220, 43221, 43222, 43223, 43224, 43227, 43228, 43229, 43230, 43231, 43235, 43240]
i=0

cols=['zipcode','neighborhood','population','population density','median-income']
df=pd.DataFrame(columns=cols)
for zipz in zipcodes:
  idx=zipcodes[i]
  url = f'https://www.unitedstateszipcodes.org/{idx}/'
  soup = Soup.get(url)
  boro=soup.find("dd",mode="first")
  boro=boro.text
  table = soup.find("table", {"class": "table table-hover"},mode='all')
  table=str(table)
  big_table=table.split(sep='<tr>',maxsplit=15)
  pop=big_table[1]
  pop=pop.split(sep=">",maxsplit=5)
  pop=pop[3]
  pop=pop.split(sep="<")
  pop=pop[0]
  pop_dens=big_table[2]
  pop_dens=pop_dens.split(sep=">",maxsplit=5)
  pop_dens=pop_dens[3]
  pop_dens=pop_dens.split(sep="<")
  pop_dens=pop_dens[0]
  try:
    med_income=big_table[8]
    med_income=med_income.split(sep=">",maxsplit=5)
    med_income=med_income[3]
    med_income=med_income.split(sep="<")
    med_income=med_income[0]
  except:
    med_income=None
  row=[zipcodes[i],boro,pop,pop_dens,med_income]
  df.loc[i]=row
  i+=1

df



  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gazpacho: filename=gazpacho-1.1-cp37-none-any.whl size=7482 sha256=a89135b0f65b63458188744fe5d4e4aa2f2600b826e2b95cc2ece5f0e72901e2
  Stored in directory: /root/.cache/pip/wheels/86/01/3d/a957187059b566e5a535d59dbeb1dfceb69330e3626dfc90ce
Successfully built gazpacho


,zipcode,neighborhood,population,population density,median-income
0,43002,"Amlin, OH","2,262","2,275","$85,338"
1,43004,"Blacklick, OH","22,727","1,757","$71,060"
2,43016,"Dublin, OH","31,306","1,731","$86,446"
3,43017,"Dublin, OH","37,626","2,229","$94,704"
4,43026,"Hilliard, OH","54,017","1,540","$76,813"
5,43035,"Lewis Center, OH","24,721","1,262","$101,295"
6,43054,"New Albany, OH","20,566","1,025","$103,491"
7,43065,"Powell, OH","38,821","1,344","$110,031"
8,43081,"Westerville, OH","55,991","2,274","$68,306"
9,43082,"Westerville, OH","29,946","1,354","$110,137"


In [2]:
df['median-income'][22]='$17,137' #manually entered this one
df.describe
#Now let's find longitude and latitude by zipcode
url = 'https://gist.githubusercontent.com/erichurst/7882666/raw/5bdc46db47d9515269ab12ed6fb2850377fd869e/US%2520Zip%2520Codes%2520from%25202013%2520Government%2520Data'
long_lat=pd.read_csv(url)
long_lat.columns=['zipcode','latitude','longitude']
merged_df = pd.merge(df, long_lat, on="zipcode")
merged_df.describe
#Let's limit the neighborhood to what is truly centered in Columbus (since it is such a large city), so it will now be 28 rows
new_df=merged_df[merged_df['neighborhood']=='Columbus, OH'].reset_index(drop=True)
new_df.describe


<bound method NDFrame.describe of    zipcode  neighborhood population  ... median-income   latitude  longitude
0    43085  Columbus, OH     23,258  ...       $77,156  40.100924 -83.013402
1    43201  Columbus, OH     35,495  ...       $21,086  39.990830 -82.999946
2    43202  Columbus, OH     20,251  ...       $40,904  40.019812 -83.015100
3    43203  Columbus, OH      8,108  ...       $18,763  39.973084 -82.969026
4    43204  Columbus, OH     42,104  ...       $40,815  39.960662 -83.082542
5    43205  Columbus, OH     12,272  ...       $24,779  39.957019 -82.962071
6    43206  Columbus, OH     21,864  ...       $40,720  39.942452 -82.974175
7    43207  Columbus, OH     45,144  ...       $39,348  39.895590 -82.964068
8    43210  Columbus, OH      9,432  ...       $17,137  40.005435 -83.023227
9    43211  Columbus, OH     21,600  ...       $21,908  40.011814 -82.970532
10   43212  Columbus, OH     18,551  ...       $51,585  39.986564 -83.043198
11   43213  Columbus, OH     30,444  ...  

Now, let's begin clustering!

In [3]:
#Import all the libraries
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [4]:
#First let's use Folium to show us what these points look like on a map of Columbus
#The general coordinates of downtown are:39.9612° N, 82.9988° W
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab

latitude=39.96
longitude=-82.999
map_columbus = folium.Map(location=[latitude, longitude], zoom_start=11)


# add markers to map
for lat, lng, zipzz in zip(new_df['latitude'], new_df['longitude'],new_df['zipcode']):
    label = folium.Popup(zipzz, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        #popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_columbus)  
    
map_columbus

In [5]:
#FOURSQUARE

#FOURSQUARE
CLIENT_ID = 'RWLS315GSA2SN0CZKI3AF4MXUVUI1GTXRR25TYJHPMY40MVS' # your Foursquare ID
CLIENT_SECRET = 'J02G1AVJ2FQ41IXMKWEW0PDQ0GKPD1UIZWMVCKAI0L1WHPPW' # your Foursquare Secret
VERSION = '20210626' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value
#merged_df.loc[0, 'Neighborhood']
neighborhood_latitude = new_df.loc[0, 'latitude'] # neighborhood latitude value
neighborhood_longitude = new_df.loc[0, 'longitude'] # neighborhood longitude value

neighborhood_zipcode = new_df.loc[0, 'zipcode'] # neighborhood name

def getNearbyVenues(zipcodes, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for zipcodes, lat, lng in zip(zipcodes, latitudes, longitudes):
        print(zipcodes)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            zipcodes, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['zipcodes', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

columbus_venues = getNearbyVenues(zipcodes=new_df['zipcode'],
                                   latitudes=new_df['latitude'],
                                   longitudes=new_df['longitude']
                                  )


43085
43201
43202
43203
43204
43205
43206
43207
43210
43211
43212
43213
43214
43215
43217
43219
43220
43221
43222
43223
43224
43227
43228
43229
43230
43231
43235
43240


In [6]:
# one hot encoding
columbus_onehot = pd.get_dummies(columbus_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
columbus_onehot['zipcodes'] = columbus_venues['zipcodes'] 

# move neighborhood column to the first column
fixed_columns = [columbus_onehot.columns[-1]] + list(columbus_onehot.columns[:-1])
columbus_onehot = columbus_onehot[fixed_columns]

columbus_onehot.head()

columbus_grouped = columbus_onehot.groupby('zipcodes').mean().reset_index()
columbus_grouped

,zipcodes,Accessories Store,Airport,American Restaurant,Arcade,Asian Restaurant,Automotive Shop,BBQ Joint,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Beach Bar,Big Box Store,Bookstore,Botanical Garden,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Business Service,Butcher,Cajun / Creole Restaurant,Camera Store,Candy Store,Chinese Restaurant,Chiropractor,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Basketball Court,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Country Dance Club,Deli / Bodega,Department Store,Diner,Discount Store,Dive Bar,Dive Spot,Electronics Store,Fast Food Restaurant,Financial or Legal Service,Fish & Chips Shop,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Fried Chicken Joint,Furniture / Home Store,Gas Station,Gastropub,Gift Shop,Golf Course,Grocery Store,Gym,Home Service,Hookah Bar,Hotel,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Kids Store,Kitchen Supply Store,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Motorcycle Shop,Movie Theater,Museum,Music Venue,Noodle House,Optical Shop,Outdoor Sculpture,Paper / Office Supplies Store,Park,Peruvian Restaurant,Pet Store,Pharmacy,Picnic Area,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Record Shop,Rental Service,Restaurant,Rock Club,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Snack Place,Spa,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Thrift / Vintage Store,Tourist Information Center,Toy / Game Store,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,43085,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000,0.000000,0.0,0.000000,0.000000,0.0,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,43201,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,

In [7]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['zipcodes']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
columbus_venues_sorted = pd.DataFrame(columns=columns)
columbus_venues_sorted['zipcodes'] = columbus_grouped['zipcodes']

for ind in np.arange(columbus_grouped.shape[0]):
    columbus_venues_sorted.iloc[ind, 1:] = return_most_common_venues(columbus_grouped.iloc[ind, :], num_top_venues)

columbus_venues_sorted.head()

,zipcodes,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,43085,Playground,Restaurant,Furniture / Home Store,Concert Hall,Construction & Landscaping,Convenience Store,Comic Shop,Cosmetics Shop,Fast Food Restaurant,Country Dance Club
1,43201,Motorcycle Shop,Video Store,Lounge,Supermarket,Women's Store,Diner,Electronics Store,Dive Spot,Dive Bar,Discount Store
2,43202,Gas Station,Coffee Shop,Park,Mediterranean Restaurant,Bubble Tea Shop,Grocery Store,Pizza Place,Pub,Brewery,Salon / Barbershop
3,43203,Park,Plaza,Chinese Restaurant,Cajun / Creole Restaurant,Music Venue,Sandwich Place,Bank,Deli / Bodega,Department Store,Electronics Store
4,43204,Park,Gym,Department Store,Fast Food Restaurant,Electronics Store,Dive Spot,Dive Bar,Discount Store,Diner,Women's Store


In [9]:

# set number of clusters
kclusters = 10

columbus_grouped_clustering = columbus_grouped.drop('zipcodes', 1)
columbus_grouped_clustering.head()
#columbus_venues_sorted.drop(['Cluster Labels'], axis=1, inplace=True)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(columbus_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 
# add clustering labels
columbus_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

columbus_merged = new_df

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
columbus_merged = columbus_merged.join(columbus_venues_sorted.set_index('zipcodes'), on='zipcode')
columbus_merged.dropna(axis=0,inplace=True)

columbus_merged.head() # check the last columns!
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(columbus_merged['latitude'], columbus_merged['longitude'], columbus_merged['zipcode'], columbus_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Clustering is finished! Now let's interpret the data.

In [11]:
import statistics as st
#columbus_merged.head()
cluster0=columbus_merged[columbus_merged['Cluster Labels']==0.0].reset_index(drop=True)
cluster1=columbus_merged[columbus_merged['Cluster Labels']==1.0].reset_index(drop=True)
cluster2=columbus_merged[columbus_merged['Cluster Labels']==2.0].reset_index(drop=True)
cluster3=columbus_merged[columbus_merged['Cluster Labels']==3.0].reset_index(drop=True)
cluster4=columbus_merged[columbus_merged['Cluster Labels']==4.0].reset_index(drop=True)
cluster5=columbus_merged[columbus_merged['Cluster Labels']==5.0].reset_index(drop=True)
cluster6=columbus_merged[columbus_merged['Cluster Labels']==6.0].reset_index(drop=True)
cluster7=columbus_merged[columbus_merged['Cluster Labels']==7.0].reset_index(drop=True)
cluster8=columbus_merged[columbus_merged['Cluster Labels']==8.0].reset_index(drop=True)
cluster9=columbus_merged[columbus_merged['Cluster Labels']==9.0].reset_index(drop=True)
clusterz=[cluster0,cluster1,cluster2,cluster3,cluster4,cluster5,cluster6,cluster7,cluster8,cluster9]
collies=['cluster','min_income','max_income','max pop', 'min pop','max pop_dens','min pop_dens','most common venue','second mostcommon','length']
clustericious=pd.DataFrame(columns=collies)
i=0
for clusterzz in clusterz: 
  min_income=min(clusterz[i]['median-income'])
  max_income=max(clusterz[i]['median-income'])
  max_pop=max(clusterz[i]['population'])
  min_pop=min(clusterz[i]['population'])
  max_pop_dens=max(clusterz[i]['population density'])
  min_pop_dens=min(clusterz[i]['population density'])
  common_venue_1=st.mode(clusterz[i]['1st Most Common Venue'])
  common_venue_2=st.mode(clusterz[i]['2nd Most Common Venue'])
  length=len(clusterz[i])
  name=f'cluster{i}'
  row=[name,min_income,max_income,max_pop,min_pop,max_pop_dens,min_pop_dens,common_venue_1,common_venue_2,length]
  clustericious.loc[i]=row
  i+=1
clustericious

,cluster,min_income,max_income,max pop,min pop,max pop_dens,min pop_dens,most common venue,second mostcommon,length
0,cluster0,"$17,137","$77,156","9,432","12,272","8,067","1,385",Sandwich Place,Park,17
1,cluster1,"$34,521","$34,521","30,444","30,444","3,381","3,381",Country Dance Club,Women's Store,1
2,cluster2,"$81,343","$81,343","31,265","31,265","3,347","3,347",Playground,Fish & Chips Shop,1
3,cluster3,"$64,666","$64,666","38,493","38,493","2,854","2,854",Golf Course,Women's Store,1
4,cluster4,"$58,199","$58,199","24,989","24,989","3,671","3,671",Shipping Store,Baseball Field,1
5,cluster5,"$39,894","$39,894","50,737","50,737","2,507","2,507",Home Service,Clothing Store,1
6,cluster6,"$39,348","$39,348","45,144","45,144","1,936","1,936",Light Rail Station,Women's Store,1
7,cluster7,"$33,598","$33,598","38,699","38,699","4,625","4,625",Candy Store,Home Service,1
8,cluster8,"$65,785","$65,785","51,161","51,161","2,445","2,445",Beach Bar,Tourist Information Center,1
9,cluster9,"$40,815","$40,815","42,104","42,104","4,679","4,679",Park,Gym,1


In [12]:
print(cluster2['zipcode'])         

0    43221
Name: zipcode, dtype: object


Result: 

The best place to set up a high-end restaurant is where the median-income is highest. This is Cluster 2, which is zipcode 43221. This area is also called Upper Arlington or Hilliard, and it is a part of Columbus, OH. We also know that the population size for this area is substantial, and the lower population-density is most likely due to wealthier individuals owning more land. We also know a British restaurant would be successful here because the second most common venue is a "Fish and Chips Shop."